In [1]:
experiment_name = 'runningExample'

In [2]:
import pm4py
import itertools
import time
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "1234"))

lbl_n_logs        = 'number of logs'
lbl_n_events      = 'number of events'
lbl_n_activities  = 'number of activities'
lbl_n_observed    = 'number of observed'
lbl_n_objects     = 'number of objects'
lbl_n_df          = 'number of df'
lbl_n_corr        = 'number of corr'

lbl_filter_df_time = 'filter on event time'
lbl_filter_df_object_type = 'filter on object types'
lbl_filter_df_object_type_time = 'filter on object types and time'
lbl_filter_df_objects = 'filter on objects'
lbl_filter_df_objects_time = 'filter on objects and time'


def get_from_neo4j(cmd):
    res = graph.run(cmd).data()[0]
    return res


def get_neo4j_summary(res = {}):
    start_time = time.time()
    measure_name = lbl_n_logs
    res[measure_name]     = get_from_neo4j('MATCH(n:Log) return count(n) as result')['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    
    start_time = time.time()
    measure_name = lbl_n_events
    res[measure_name]     = get_from_neo4j('MATCH(n:Event) return count(n) as result')['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    start_time = time.time()
    measure_name = lbl_n_df
    res[measure_name]     = get_from_neo4j("match ()-[r:DF]->() return count(r) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    
    start_time = time.time()
    measure_name = lbl_n_activities
    res[measure_name]     = get_from_neo4j("MATCH(n:Class) return count(n) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    
    start_time = time.time()
    measure_name = lbl_n_objects
    res[measure_name]     = get_from_neo4j("MATCH(n:Entity) WHERE not (n)-[:REL {Type:'Reified'}]->() return count(n) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    start_time = time.time()
    measure_name = lbl_n_observed
    res[measure_name]     = get_from_neo4j("MATCH ()-[n:OBSERVED]->() return count(n) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    
    start_time = time.time()
    measure_name = lbl_n_corr
    res[measure_name]    = get_from_neo4j("MATCH(n1:Event)-[r1:CORR]->(n2:Entity) WHERE not (n2)-[:REL {Type:'Reified'}]->()return count(r1) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    return res



def get_neo4j_filter(res = {}):
    start_time = time.time()
    measure_name = lbl_filter_df_time
    res[measure_name]    = get_from_neo4j("MATCH (n1:Event)-[r:DF]->(n2:Event) where n1.timestamp>datetime('1981-01-01T00:00:00+01:00') and n1.timestamp<datetime('1982-01-01T00:00:00+01:00') and n2.timestamp>datetime('1981-01-01T00:00:00+01:00') and n2.timestamp<datetime('1982-01-01T00:00:00+01:00') return count(r) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
    
    
    start_time = time.time()
    measure_name = lbl_filter_df_object_type
    res[measure_name]    = get_from_neo4j("MATCH (n1:Event)-[r1:DF]->(n2:Event)-[r2:CORR]->(:Entity {EntityType:'xxxxxxxxxxxxxxxxxx'}) return count(r1) as result")['result']
    res[measure_name+ ' (Time)']     = time.time() - start_time
        
    start_time = time.time()
    measure_name = lbl_filter_df_object_type_time
    res[measure_name]    = get_from_neo4j("MATCH (n1:Event)-[r1:DF]->(n2:Event)-[r2:CORR]->(:Entity {EntityType:'xxxxxxxxxxxxxxxxxx'}) where n1.timestamp>datetime('1981-01-01T00:00:00+01:00') and n1.timestamp<datetime('1982-01-01T00:00:00+01:00') and n2.timestamp>datetime('1981-01-01T00:00:00+01:00') and n2.timestamp<datetime('1982-01-01T00:00:00+01:00') return count(r1) as result")['result'] 
    res[measure_name+ ' (Time)']     = time.time() - start_time
        
    start_time = time.time()
    measure_name = lbl_filter_df_objects
    res[measure_name]    = get_from_neo4j("MATCH (ent1:Entity)<-[r1:CORR]-(e1:Event)-[r2:DF]->(e2:Event)-[r3:CORR]->(ent2:Entity) where ent1.uID in ['xxxx', 'xxxxxxx'] and ent2.uID in ['xxxx', 'xxxxxxx'] return count(r2) as result")['result'] 
    res[measure_name+ ' (Time)']     = time.time() - start_time
    res[measure_name+ ' (Time)']     = time.time() - start_time
        
    start_time = time.time()
    measure_name = lbl_filter_df_objects_time
    res[measure_name]    = get_from_neo4j("MATCH (ent1:Entity)<-[r1:CORR]-(e1:Event)-[r2:DF]->(e2:Event)-[r3:CORR]->(ent2:Entity) where ent1.uID in ['xxxx', 'xxxxxxx'] and ent2.uID in ['xxxx', 'xxxxxxx'] and e1.timestamp>datetime('1981-01-01T00:00:00+01:00') and e1.timestamp<datetime('1982-01-01T00:00:00+01:00') and e2.timestamp>datetime('1981-01-01T00:00:00+01:00') and e2.timestamp<datetime('1982-01-01T00:00:00+01:00') return count(r2) as result")['result'] 
    res[measure_name+ ' (Time)']     = time.time() - start_time

    return res


In [3]:
res = get_neo4j_summary()

In [4]:
res = get_neo4j_filter(res)

In [5]:
with open('./experimental_results/result.csv', 'a') as f_object:
 
    for  i,v in res.items():
        f_object.write('neo4j,{},{},{}\n'.format(experiment_name,i,v))        
        
    f_object.close()